# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [33]:
import pandas as pd
import numpy as np
from scipy import stats

In [34]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [35]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [36]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit



# 1. What test is appropriate for this problem? Does CLT apply?#

Methodology for choosing a test: 
![alt text](https://bloomingtontutors.com/blog/images/1/4/d/a/b/x14dab4d068c4ea2fa866a1b19e3c2f2dabb3c174-z-vs-t-distribution-flowchart.jpeg.pagespeed.ic.OpVrQ27n1N.webp "https://bloomingtontutors.com/blog/when-to-use-the-z-test-versus-t-test")
    
We should use a proportional test since we will be comparing the rates of callbacks based on race, defined as white call backs divided by black call backs.
    
> From [investopedia.com:](https://www.investopedia.com/terms/c/central_limit_theorem.asp)
    
> The central limit theorem states that the distribution of sample means approximates a normal distribution as the sample size gets larger (assuming that all samples are identical in size), regardless of population distribution shape. 
   
> As a general rule, sample sizes equal to or greater than 30 are considered sufficient for the CLT to hold, meaning the distribution of the sample means is fairly normally distributed. So, the more samples that one takes results in the graph looking more and more like that of a normal distribution.

Since the sample sizes are greater than 30, the CLT holds.

# 2. What are the null and alternate hypotheses?#
Ho: white_rate = black_rate <br>
Ha: white_rate != black_rate    

In [37]:
black_applicants = len(data[data.race=='b'])
white_applicants = len(data[data.race=='w'])
total_applicants = len(data)
black_callbacks = sum(data[data.race=='b'].call)
white_callbacks = sum(data[data.race=='w'].call)
black_rate = black_callbacks/black_applicants
white_rate = white_callbacks/white_applicants

print('Black applicants:',str(black_applicants))
print('White applicants:',str(white_applicants))
print('Total applicants:',str(total_applicants))
print('Black call backs:',str(black_callbacks))
print('White call backs:',str(white_callbacks))
print('Black call back rate:',str(black_rate))
print('White call back rate:',str(white_rate))
print('Is',str(total_applicants*white_rate/black_rate),'> 10?')





Black applicants: 2435
White applicants: 2435
Total applicants: 4870
Black call backs: 157.0
White call backs: 235.0
Black call back rate: 0.06447638603696099
White call back rate: 0.09650924024640657
Is 7289.490445859872 > 10?


Since np = 7289.5 > 10, we will use a z-test

# 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches. #
   
p = (white_rate * white_applicants + black_rate * black_applicants) / (white_applicants + black_applicants) <br>
Let's set the significance level to 0.01. 

<br>
Following methodology here:


[Two-sided test for difference between proportions](https://stattrek.com/hypothesis-test/difference-in-proportions.aspx?Tutorial=AP)

In [44]:
p = (white_rate * white_applicants + black_rate * black_applicants) / (white_applicants + black_applicants)
SE = np.sqrt(p*(1-p)*(1/white_applicants+1/black_applicants))
z = (white_rate - black_rate)/SE
p_value = stats.norm.sf(abs(z))*2 #twosided
   
print('Weighted average of call-back rates:',str(p))
print('P-value:',str(p_value))    
print('SE: +/-',str(SE))
print('Z',str(z))

Weighted average of call-back rates: 0.08049281314168377
P-value: 3.983886837585077e-05
SE: +/- 0.007796894036170457
Z 4.108412152434346


Since p-value of 3.98e-05 < 0.01, we reject the null hypothesis with 99% confidence.
 
 
 


# 4. Write a story describing the statistical significance in the context or the original problem.


Since we rejected the null hypothesis which stated that white call back rates were equal to black call back rates, and we used a significance level of 0.01, we are 99% confident that resumes coming from applicants with white-sounding names are more likely to be hired than applicants who apply with black-sounding names.

The average call-back rate is 8 +/- 0.78%. The black call back rate is 6.45%, below the confidence interval with p-value of 3.984e-05, and the white call back rate is 9.65%, above the the confidence interval with p-value of 3.984e-05.


# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis does not mean that race or name is the most important factor in callback success. There are other factors that were not explored, such as educational attainment, years of experience, type of experience, number of jobs held, and private/nonprofit/public job type. We can perform similar tests with respect to these factors to determine if they significant affect callback success.